In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

%matplotlib inline

In [64]:
def get_out_file_names(folder:str)->list:
    return glob.glob(f"{folder}/*.out", recursive=True)

def get_csv_file_names(folder:str)->list:
    return glob.glob(f"{folder}/*.csv", recursive=True)

def get_basic_data(filename:str)->tuple:
    fields = ["Best Distance: ", "Process Time: ", "Wall Time: "]
    t = []
    with open(filename, "r") as f:
        lines = [line.strip() for line in f.readlines()]
        for field in fields:
            for line in lines:
                if line.startswith(field):
                    t.append(float(line[len(field):]))
    return tuple(t)

def get_out_df(filenames:list, is_cached:bool)->pd.DataFrame:
    insts = ["inst-0", "inst-13", "inst-5"]
    variants = ["base", "variant1", "variant2"]
    runs = [1, 2, 3, 4, 5]
    df_dict = {
        "cached": [],
        "instance": [],
        "variant": [],
        "run_number": [],
        "best_distance": [],
        "process_time": [],
        "wall_time": [],
    }
    for inst in insts:
        for variant in variants:
            for run in runs:
                filepat = f"{inst}.{variant}.{run}.out"
                for fname in filenames:
                    if filepat in fname:
                        bd, pt, wt = get_basic_data(fname)
                        df_dict['cached'].append(is_cached)
                        df_dict['instance'].append(inst)
                        df_dict['variant'].append(variant)
                        df_dict['run_number'].append(run)
                        df_dict['best_distance'].append(bd)
                        df_dict['process_time'].append(pt)
                        df_dict['wall_time'].append(wt)
    return pd.DataFrame(df_dict)

    
def test():
    f = get_out_file_names('cache')
    df = get_out_df("None", f, True)
    print(df.describe().T)

def get_all_out_files_data_as_dataframe():
    cf = get_out_file_names('cache')
    ncf = get_out_file_names('nocache')
    cdf = get_out_df(cf, True)
    ncdf = get_out_df(ncf, False)
    return cdf, ncdf

def get_description_df(cdf:pd.DataFrame, ncdf:pd.DataFrame):
    df = pd.concat([cdf, ncdf])
    print(df.shape, cdf.shape, ncdf.shape)
    bdf = df[['cached', 'instance', 'variant', 'best_distance']]
    pdf = df[['cached', 'instance', 'variant', 'process_time']]
    bdf = bdf.groupby(by=['instance', 'cached', 'variant']).describe()
    pdf = pdf.groupby(by=['instance', 'cached', 'variant']).describe()
    full_df = df[['cached', 'instance', 'variant', 'best_distance', 'process_time']].\
            groupby(by=['instance', 'cached', 'variant']).describe()
    return full_df, bdf, pdf

cdf, ncdf = get_all_out_files_data_as_dataframe()
full_description, best_distance_description, process_time_description = get_description_df(cdf, ncdf)

(90, 7) (45, 7) (45, 7)


In [67]:
best_distance_description

best_distance                              \
                                 count          mean           std   
instance cached variant                                              
inst-0   False  base               5.0  3.483211e+06  27401.561743   
                variant1           5.0  3.560954e+06  41682.079138   
                variant2           5.0  3.594154e+06  19554.124732   
         True   base               5.0  3.483211e+06  27401.561743   
                variant1           5.0  3.560954e+06  41682.079138   
                variant2           5.0  3.594154e+06  19554.124732   
inst-13  False  base               5.0  6.324916e+06  28343.936094   
                variant1           5.0  6.400748e+06  27786.048108   
                variant2           5.0  6.441843e+06  49424.881214   
         True   base               5.0  6.324916e+06  28343.936094   
                variant1           5.0  6.400748e+06  27786.048108   
                variant2           5.0  6.441843e+06  49424.881214   
inst-5   False  base               5.0  1.115341e+07  36307.619408   
                variant1           5.0  1.135044e+07  54096.755444   
                variant2           5.0  1.143268e+07  74494.355171   
         True   base               5.0  1.115341e+07  36307.619408   
                variant1           5.0  1.135044e+07  54096.755444   
                variant2           5.0  1.143268e+07  74494.355171   

                                                                    \
                                   min           25%           50%   
instance cached variant                                              
inst-0   False  base      3.452940e+06  3.461627e+06  3.486302e+06   
                variant1  3.504536e+06  3.531718e+06  3.579031e+06   
                variant2  3.564625e+06  3.588953e+06  3.594459e+06   
         True   base      3.452940e+06  3.461627e+06  3.486302e+06   
                variant1  3.504536e+06  3.531718e+06  3.579031e+06   
                variant2  3.564625e+06  3.588953e+06  3.594459e+06   
inst-13  False  base      6.305152e+06  6.309133e+06  6.312294e+06   
                variant1  6.364196e+06  6.388281e+06  6.394371e+06   
                variant2  6.397159e+06  6.417805e+06  6.425238e+06   
         True   base      6.305152e+06  6.309133e+06  6.312294e+06   
                variant1  6.364196e+06  6.388281e+06  6.394371e+06   
                variant2  6.397159e+06  6.417805e+06  6.425238e+06   
inst-5   False  base      1.109191e+07  1.115708e+07  1.116426e+07   
                variant1  1.130291e+07  1.131195e+07  1.132049e+07   
                variant2  1.133178e+07  1.140755e+07  1.142711e+07   
         True   base      1.109191e+07  1.115708e+07  1.116426e+07   
                variant1  1.130291e+07  1.131195e+07  1.132049e+07   
                variant2  1.133178e+07  1.140755e+07  1.142711e+07   

                                                      
                                   75%           max  
instance cached variant                               
inst-0   False  base      3.493055e+06  3.522131e+06  
                variant1  3.582463e+06  3.607023e+06  
                variant2  3.607021e+06  3.615712e+06  
         True   base      3.493055e+06  3.522131e+06  
                variant1  3.582463e+06  3.607023e+06  
                variant2  3.607021e+06  3.615712e+06  
inst-13  False  base      6.323959e+06  6.374044e+06  
                variant1  6.425151e+06  6.431742e+06  
                variant2  6.444003e+06  6.525008e+06  
         True   base      6.323959e+06  6.374044e+06  
                variant1  6.425151e+06  6.431742e+06  
                variant2  6.444003e+06  6.525008e+06  
inst-5   False  base      1.116558e+07  1.118821e+07  
                variant1  1.139532e+07  1.142153e+07  
                variant2  1.146176e+07  1.153519e+07  
         True   base      1.116558e+07  1.118821e+07  
                variant1  1.139532e+07

In [61]:
cdf, ncdf = get_all_out_files_data_as_dataframe()

In [62]:
cdf

,cached,instance,variant,run_number,best_distance,process_time,wall_time
0,True,inst-0,base,1,3.452940e+06,22.625000,22.652222
1,True,inst-0,base,2,3.493055e+06,21.671875,21.744243
2,True,inst-0,base,3,3.486302e+06,23.359375,23.409590
3,True,inst-0,base,4,3.461627e+06,22.765625,22.763834
4,True,inst-0,base,5,3.522131e+06,22.500000,22.509611
5,True,inst-0,variant1,1,3.582463e+06,8.359375,8.367596
6,True,inst-0,variant1,2,3.531718e+06,9.812500,9.843245
7,True,inst-0,variant1,3,3.504536e+06,9.453125,9.450127
8,True,inst-0,variant1,4,3.607023e+06,10.015625,10.034549
9,True,inst-0,variant1,5,3.579031e+06,11.000000,11.059918


In [ ]:
c